<a href="https://colab.research.google.com/github/Jellyjellyjinjin/dacon-multimodal-vqa/blob/main/LLaVA_zeroshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

clone the LLaVA

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
%cd /content/LLaVA

Install

In [ ]:
!pip install --upgrade pip
!pip install -e .
!pip install ninja
!pip install flash-attn --no-build-isolation

load question

In [ ]:
import csv
import json

with open('/content/test.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    data = list(reader)

json_data = []
for row in data:
    id, image_id, question = row
    json_data.append(question)

Input_question.jsonl (json to jsonl)

In [ ]:
jsonl_output_file = "/content/test10.jsonl"

# JSON 데이터를 JSONL 형식으로 변환하여 파일에 저장
with open(jsonl_output_file, "w") as file:
    for obj in json_data:
        # 개별 JSON 객체를 개행(\n)으로 구분하여 적기.
        json.dump(obj, file)
        file.write("\n")

# Inference

In [ ]:
!python3 /content/LLaVA/llava/eval/model_vqa.py \
    --model-path liuhaotian/llava-v1-0719-336px-lora-merge-vicuna-13b-v1.3\
    --question-file \
    /content/test13.jsonl \
    --image-folder \
   /content/image/test \
    --answers-file \
    /content/result14.jsonl

question file load

In [117]:
import csv
import json

with open('/content/test.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    data = list(reader)

json_data2 = []
for row in data:
    id, image_id, question = row
    json_data2.append(question)

select 8000 question, answer




In [ ]:
#question
json_data2 = json_data2[:8000]
#answer
file_path = '/content/result12.jsonl'
start_line = 8001  # 시작할 줄 번호
answer = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, start=1):
        if line_number == start_line:
            break
        print(line_number)
        info = json.loads(line)
        answer.append(info['text'].strip())


**Check to convert color and number**

In [120]:
#색상검사
def color(sentence, target_words):
    # 문장을 소문자로 변환하여 대소문자 무시
    sentence = sentence.lower()

    # 타겟 단어 리스트에서 하나씩 검사
    for target_word in target_words:
        if target_word in sentence:
          return target_word  # 문장에서 단어를 찾으면 해당 단어를 반환하고 함수 종료
    return sentence  # 문장에서 단어를 찾지 못하면 원래 문장을 반환


#숫자검사
def num(sentence, target_words):
    # 문장을 소문자로 변환하여 대소문자 무시
    sentence = sentence.lower()

    # 타겟 단어 리스트에서 하나씩 검사
    for target_word in target_words:
        if target_word in sentence:
          return target_word  # 문장에서 단어를 찾으면 해당 단어를 반환하고 함수 종료
    return sentence  # 문장에서 단어를 찾지 못하면 원래 문장을 반환


rainbow_colors = ["black","white","red", "orange", "yellow", "green","blue","purple","beige","pink","gold","brown"]
numword = ["zero","one","two","three","four", "five","six", "seven", "eight","nine","many"]


**1. Devide question**
1.   What color?  ㅡ>   color ()
2.   How many? ㅡ>  num()



In [121]:
def process_sentence(question, answer):
    if question.startswith("What color"):
        # "What color"로 시작하는 경우
        return color(answer, rainbow_colors)

    elif question.startswith("How many"):
        # "How many"로 시작하는 경우
        return num(answer, numword)

    else:
        # 위 두 경우에 해당하지 않는 경우
        return answer



# 모든 질문과 답변에 대한 처리
processed_answers = [process_sentence(question, answer) for question, answer in zip(json_data2, answer)]


**2.Remove stopword**

In [122]:
import re

def remove_articles(text):

    # 소문자로 변환
    text = text.lower()

    # 정규 표현식을 사용하여 관사 제거
    text = re.sub(r'\b(a|an|the|in|on|at|by|with|from|)\b', '', text)

    # 여러 개의 공백을 단일 공백으로 대체
    text = re.sub(r'\s+', ' ', text).strip()

    # 선택된 단어를 다시 결합
    result_text = ''.join(text)

    return result_text



**3.remove duplicate words from the question in the answer**

In [124]:
def remove_words_from_b_if_in_a(sentence_a, sentence_b):
    # 문장 A와 B를 공백을 기준으로 단어로 분리
    sentencea_a = sentence_a[:-1]
    words_a = sentencea_a.split()
    words_b = sentence_b.split()

    if all(word in 'or' and word in 'and' and word in 'many' for word in words_a):
      df2 = [i for i  in words_b if i not in words_a]
      return df2
    else:
      return sentence_b


processed_answers = [remove_articles(answer) for answer in  processed_answers]

# load Keybert

In [13]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=d9c9ffa55bc7539d8c13539fba1a6b82a981653e8805eec69b74c2c35df6d434
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=5ff11349aea5d5ab002cb34a3bb083caee9ce73583f94ca156e6a7249abf8aee
 

In [15]:
from sentence_transformers import SentenceTransformer, util


In [16]:
from sentence_transformers import SentenceTransformer, util
from keybert import KeyBERT

kw_model = KeyBERT()

keybert 로 question과 유사도가 높은 answer 키워드 추출

In [126]:
from tqdm import tqdm

plzan = []

for qdoc, adoc in tqdm(zip(json_data2, processed_answers)):
    if " and " in adoc:
      if len(adoc.split()) >= 5:
        adoc = remove_words_from_b_if_in_a(qdoc,adoc)
        qdoc = qdoc[:3]
        keywords = kw_model.extract_keywords(qdoc)
        a = kw_model.extract_keywords(adoc, keyphrase_ngram_range=(1,1), stop_words=None)
        plzan.append(a[0][0])
      else :
        plzan.append(adoc)
    else :
      adoc = remove_words_from_b_if_in_a(qdoc,adoc)
      qdoc = qdoc[:3]
      keywords = kw_model.extract_keywords(qdoc)
      a = kw_model.extract_keywords(adoc, keyphrase_ngram_range=(1,1), stop_words=None)
      plzan.append(a[0][0])

8000it [03:15, 40.98it/s]


In [127]:
print(plzan[:10])

['no', 'beach', 'red', 'white', 'yes', 'black', 'two', 'yes', 'yes', 'blue']


**change string to int**

In [128]:
number_words_dict = {
    "zero": 0,
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
}
def convert_word_to_number(text):
    words = text.lower().split()
    for word in words:
        if word in number_words_dict:
            return str(number_words_dict[word])
    return text


processed_answers = [convert_word_to_number( answerone) for answerone in plzan]

In [129]:
processed_answers[:10]

['no', 'beach', 'red', 'white', 'yes', 'black', '2', 'yes', 'yes', 'blue']

In [131]:
import pandas as pd

processed_answers = pd.DataFrame(processed_answers)
processed_answers.to_csv('llava.csv', index=False)